In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import hour, mean, round
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# Read in the csv
from pyspark import SparkFiles
url = "https://raw.githubusercontent.com/ejmatthe/project-four/857fe47338c94638dc95a4d6cc5d17b2e945f272/resources/master.csv"
spark.sparkContext.addFile(url)
suicides_df = spark.read.csv(SparkFiles.get("master.csv"), sep=",", header=True)

# Print head to confirm it was read in
suicides_df.show()

+-------+----+------+-----------+-----------+----------+---------------------+----------------+------------------+---------------+------------+
|country|year|   sex|        age|suicides_no|population|suicides_per_100k_pop|gdp_for_year_USD|gdp_per_capita_USD|     generation|suicide_tier|
+-------+----+------+-----------+-----------+----------+---------------------+----------------+------------------+---------------+------------+
|Albania|1987|  male|15-24 years|         21|    312900|                 6.71|      2156624900|               796|   Generation X|           0|
|Albania|1987|  male|35-54 years|         16|    308000|                 5.19|      2156624900|               796|         Silent|           0|
|Albania|1987|female|15-24 years|         14|    289700|                 4.83|      2156624900|               796|   Generation X|           0|
|Albania|1987|  male|  75+ years|          1|     21800|                 4.59|      2156624900|               796|G.I. Generation|      

In [5]:
# Create a temporary view of the DataFrame.
suicides_df.createOrReplaceTempView('suicides')